In [ ]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00


In [ ]:
from os.path import isfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import geemap
from geemap import ml
import ee
from google.colab import drive

In [ ]:
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cpWN7wh7DmSP0UcXPtkFmCQBAksZqK3zAn5NjMm5f9g&tc=8zxmcGta1HgILyu_2_804BisR5-MdnEt8mD1ZvLQORw&cc=FuBDg60vb9YlBMcsVi5hYoquSnmEqpZ-p4GFikB0qMs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdLIrYftqMFeyY8lr6bmmZhcXci6Bei-EcYKnarWASMQ96POnMuObytcDY8

Successfully saved authorization token.


In [ ]:
# @title Imports
modis_old = ee.ImageCollection("MODIS/061/MOD16A2GF")
modis = ee.ImageCollection("MODIS/061/MOD16A2")
train_areas_aez_1to5 = '''Load the AEZ training file provided with this repository as a featurecollection'''
humidData = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")
landsat = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
train_1 = ee.Geometry.Polygon([[[72.70889555310633, 24.81221369014938],
      [72.94510160779383, 24.814706704743912],
      [72.95883451795008, 25.011496063893],
      [72.72262846326258, 25.011496063893]]])
train_2 = ee.Geometry.Polygon([[[76.15467833543174, 27.090600068824497],
      [76.38730019358567, 27.090600984707095],
      [76.4205429506333, 27.317203162483448],
      [76.17682361066926, 27.327046399302628]]])
train_3 = ee.Geometry.Polygon([[[85.10704234177318, 20.855094827790474],
      [85.35990847245343, 20.855094850562473],
      [85.35990990576566, 21.162996015884207],
      [85.11803515181025, 21.162996037711117]]])
train_4 = ee.Geometry.Polygon([[[83.41761593569909, 19.094822071800536],
      [83.78563609604083, 19.115276509324666],
      [83.75312693783891, 19.431970781645045],
      [83.47177572032408, 19.452377993728785]]])
train_5 = ee.Geometry.Polygon([[[78.641979855122, 22.17221402498959],
      [78.960583370747, 22.162039726975802],
      [78.905651730122, 22.487250772850125],
      [78.630993526997, 22.466947326349377]]])
train_6 = ee.Geometry.Polygon([[[80.85015900448886, 24.161556413341835],
      [81.16923593349448, 24.20170500210832],
      [81.14723365640724, 24.50242032609612],
      [80.83915262507712, 24.492407765597697]]])
train_7 = ee.Geometry.Polygon([[[81.51547176302006, 21.193916691348285],
      [81.83407527864506, 21.204159573224526],
      [81.83407527864506, 21.490670847896823],
      [81.53744441927006, 21.490670847896823]]])
train_8 = ee.Geometry.Polygon([[[84.04232723177006, 23.540600905176987],
      [84.37191707552006, 23.540600905176987],
      [84.36093074739506, 23.85246151988477],
      [84.07528621614506, 23.832363994971328]]])
train_9 = ee.Geometry.Polygon([[[86.13105466315547, 24.887537380401575],
      [86.27937009284297, 24.887537380401575],
      [86.29584958503047, 25.036938109802392],
      [86.16950681159297, 25.04689170523301]]])
train_10 = ee.Geometry.Polygon([[[87.54829099128047, 25.88997036837322],
      [87.70759274909297, 25.830653420303463],
      [87.71857907721797, 26.04306781440835],
      [87.48237302253047, 26.04306781440835]]])
train_11 = ee.Geometry.Polygon([[[80.54695006056399, 27.906223123118785],
          [80.54695006056399, 27.68998744798579],
          [80.85456724806399, 27.68998744798579],
          [80.85456724806399, 27.906223123118785]]])

In [ ]:
train_patches = [train_1, train_2, train_3, train_4, train_5, train_6, train_7, train_8, train_9, train_10, train_11]
for index in range(1,12):
  print("######################Starting: ", index, "#######################")
  # index = 1
  train_patch = train_patches[index]
  years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
  # year = '2000'
  for year in years:
  # for i in range(1):
    for season in range(1,5):
      print("index: ",index, " year: ", year, "  season: ", season)
    # season = 1

      # //-------THIS CODE IS CONFIGURED ACCORDING TO THE PARAMETERES SET ABOVE-------

      # // IMPORT THE TRAINING AREA GEOMETRIES AND NAMES FROM THE ASSETs

      #  train_area_list = train_patch.toList(train_patch.size(), 0);
      #  input_geom = ee.Feature(train_area_list.get(index)).geometry();
      input_geom = train_patch
      # print('input_geom:', input_geom);
      # // .geometry();
      input_name = 'train_'+str(index)

      # start = '2001-09-01'
      # end = '2001-09-30'
      if(season == 1):
        start = year+'-01-01'
        end = year+'-03-31'

      elif(season == 2):
        start = year+'-04-01'
        end = year+'-06-30'

      elif(season == 3):
        start = year+'-07-01'
        end = year+'-09-30'

      elif(season == 4):
        start = year+'-10-01'
        end = year+'-12-31'

      # // print(start, end)
      # print('input name', input_name);
      folder= 'Train_Area_'+input_name
      filename = str(year)+'_'+str(season)

      # // Extract Landsat and Modis for required Dates:
      landsatmodified = landsat.filterDate(start,end)
      modismodify = modis_old.filterDate(start,end)
      humidity = humidData.filterDate(start,end)

      # // print("humid bands: ", humidity.first());

      # // print("modis_modify :",modismodify);

      def input_add(image):
        newimg = image.set('input',input_geom)
        return newimg

      def final_update(image):
        input_geom = ee.Feature(image.get('input')).geometry()
        mygeom = image.geometry()
        ls_geom = mygeom.intersection(input_geom)
        area = ee.Number.parse(ls_geom.area())
        newimg = image.set('area',area)
        return newimg

      small_landsat = landsatmodified.map(input_add)
      chkpoint_landsat = small_landsat.map(final_update)
      on_shape_landsat = chkpoint_landsat.filter(ee.Filter.gt('area',0.0))

      def adddate(image):
        ymd = ee.String(image.id()).slice(-8)
        date = ee.String(ymd.slice(6,8))
        month = ee.String(ymd.slice(4,6))
        year = ee.String(ymd.slice(0,4))
        img1 = image.set('year',year)
        img1 = img1.set('month',month)
        img1 = img1.set('date',date)
        return img1

      def modisadd(image):
        ymd = ee.String(image.get('system:index'))
        year = ee.String(ymd.slice(0,4))
        month = ee.String(ymd.slice(5,7))
        date = ee.String(ymd.slice(8,10))
        img1 = image.set('year',year)
        img1 = img1.set('month',month)
        img1 = img1.set('date',date)
        return img1

      def humidadd(image):
        ymd = ee.String(image.id())
        year = ee.String(ymd.slice(1,5))
        month = ee.String(ymd.slice(5,7))
        date = ee.String(ymd.slice(7,9))
        img1 = image.set('year',year)
        img1 = img1.set('month',month)
        img1 = img1.set('date',date)
        return img1



      # // Landsat and modis with dates as properties
      landsat_date = on_shape_landsat.map(adddate)
      # Map.addLayer(landsat_date)

      # // print("landsat date :",landsat_date);

      def date_print(image):
        date = image.get('date')
        month = image.get('month')
        empt_fea = ee.Feature(None)
        fea = empt_fea.set('date',date)
        fea1 = fea.set('month',month)
        return fea1


      # ////// DATES OF LANDSAT
      dates = landsat_date.map(date_print)
      # // print("dates" , dates);

      modis_date = modismodify.map(modisadd)


      humid_date = humidity.map(humidadd)
      # // print('humid date',humid_date);
      # // print('modis_date',modis_date);
      one_img = modis_date.first()
      # // print("this image :",one_img)

      def clipping(image):
        newimg = image.clip(input_geom)
        return newimg

      def mean_et(image):
        meanValue = image.select('ET').reduceRegion(
          reducer= ee.Reducer.mean())
        return ee.Feature(None).set('et',meanValue)

      modis_clipped = modis_date.map(clipping)
      # // print("modis clipped :",modis_clipped);

      dates_modis = modis_clipped.map(date_print)

      # // Finding sweet point :
      # // print("dates modis :",dates_modis);
      # // print("dates landsat size :",dates.size());


      modis_dates_list = dates_modis.toList(dates_modis.size(),0)
      landsat_dates_list = dates.toList(dates.size(),0)

      mean_ET = modis_clipped.map(mean_et);


      # // print("mean :",mean_ET);

      def add_modis(image):
        newimg = image.set('mymodis',modis_date)
        return newimg

      def add_humid(image):
        newimg = image.set('myhumid',humid_date)
        return newimg

      landsat_withmodisr = landsat_date.map(add_modis)
      landsat_withmodis = landsat_withmodisr.map(add_humid)
      # // print('with modis ',landsat_withmodis);

      #  Strict_datematch = function(image){
      #    modiscoll = image.get('mymodis');
      #   modiscoll = ee.FeatureCollection(modiscoll);
      #    date_ls = ee.String((image.get('date')));
      #    month_ls = ee.String((image.get('month')));
      #    year_ls = ee.String((image.get('year')));
      #    newmodiscoll = ee.List([]);
      #    siz = modiscoll.size();

      #    iter = function(img_modis,newmodiscoll){
      #      date_modis = ee.String((img_modis.get('date')));
      #      month_modis = ee.String((img_modis.get('month')));
      #      year_modis = ee.String((img_modis.get('year')));
      #      dateclose = function(date_m,date_l,month_m,month_l,year_m,year_l){
      #       // Write the date matching algorithm here // DONE DONE
      #       // Use only ee.Algorithm.if(condition,statement1,statement2);
      #       // For conditions : always use exp1.equals(exp2) (Otherwise will always gives false because it is gee boolean object(can't be used with normal object))
      #        dm = ee.Number.parse(date_m);
      #        mm = ee.Number.parse(month_m);
      #        ym = ee.Number.parse(year_m);
      #        dl = ee.Number.parse(date_l);
      #        ml = ee.Number.parse(month_l);
      #        yl = ee.Number.parse(year_l);
      #        modisDate = ee.Date.fromYMD(ym,mm,dm);
      #        lsDate = ee.Date.fromYMD(yl,ml,dl);
      #        dateDiff = modisDate.difference(lsDate, 'day').abs();
      #       // CONFIGURABLE
      #       return dateDiff.lt(1);
      #     };
      #     return ee.Algorithms.If(dateclose(date_modis,date_ls,month_modis,month_ls,year_modis,year_ls),ee.List(newmodiscoll).add(img_modis),ee.List(newmodiscoll));
      #   }

      #    modis_close_collection = modiscoll.iterate(iter,newmodiscoll);
      #    newimg = image.set('closemodis',modis_close_collection);
      #   return newimg;
      # }

      def datematch(image):
        modiscoll = image.get('mymodis')
        modiscoll = ee.FeatureCollection(modiscoll)
        date_ls = ee.String((image.get('date')))
        month_ls = ee.String((image.get('month')))
        year_ls = ee.String((image.get('year')))
        newmodiscoll = ee.List([])
        siz = modiscoll.size()
        def iter(img_modis,newmodiscoll):
          date_modis = ee.String((img_modis.get('date')))
          month_modis = ee.String((img_modis.get('month')))
          year_modis = ee.String((img_modis.get('year')))
          def dateclose(date_m,date_l,month_m,month_l,year_m,year_l):
            # // Write the date matching algorithm here // DONE DONE
            # // Use only ee.Algorithm.if(condition,statement1,statement2);
            # // For conditions : always use exp1.equals(exp2) (Otherwise will always gives false because it is gee boolean object(can't be used with normal object))
            dm = ee.Number.parse(date_m)
            mm = ee.Number.parse(month_m)
            ym = ee.Number.parse(year_m)
            dl = ee.Number.parse(date_l)
            ml = ee.Number.parse(month_l)
            yl = ee.Number.parse(year_l)
            modisDate = ee.Date.fromYMD(ym,mm,dm)
            lsDate = ee.Date.fromYMD(yl,ml,dl)
            dateDiff = modisDate.difference(lsDate, 'day')
            # // CONFIGURABLE
            datebool = dateDiff.lte(6) and(dateDiff.gte(0))
            return datebool
          return ee.Algorithms.If(dateclose(date_modis,date_ls,month_modis,month_ls,year_modis,year_ls),ee.List(newmodiscoll).add(img_modis),ee.List(newmodiscoll))

        modis_close_collection = modiscoll.iterate(iter,newmodiscoll)
        newimg = image.set('closemodis',modis_close_collection)
        return newimg

      def datematch_humid(image):

        modiscoll = image.get('myhumid')
        modiscoll = ee.FeatureCollection(modiscoll)
        date_ls = ee.String((image.get('date')))
        month_ls = ee.String((image.get('month')))
        year_ls = ee.String((image.get('year')))
        newmodiscoll = ee.List([])
        siz = modiscoll.size()

        def iter(img_modis,newmodiscoll):
          date_modis = ee.String((img_modis.get('date')))
          month_modis = ee.String((img_modis.get('month')))
          year_modis = ee.String((img_modis.get('year')))
          def dateclose(date_m,date_l,month_m,month_l,year_m,year_l):
              # // Write the date matching algorithm here // DONE DONE
              # // Use only ee.Algorithm.if(condition,statement1,statement2);
              # // For conditions : always use exp1.equals(exp2) (Otherwise will always gives false because it is gee boolean object(can't be used with normal object))
              dm = ee.Number.parse(date_m)
              mm = ee.Number.parse(month_m)
              ym = ee.Number.parse(year_m)
              dl = ee.Number.parse(date_l)
              ml = ee.Number.parse(month_l)
              yl = ee.Number.parse(year_l)
              modisDate = ee.Date.fromYMD(ym,mm,dm)
              lsDate = ee.Date.fromYMD(yl,ml,dl)
              dateDiff = modisDate.difference(lsDate, 'day').abs()
              # // CONFIGURABLE
              return dateDiff.lt(1)

          return ee.Algorithms.If(dateclose(date_modis,date_ls,month_modis,month_ls,year_modis,year_ls),ee.List(newmodiscoll).add(img_modis),ee.List(newmodiscoll))

        modis_close_collection = ee.FeatureCollection(modiscoll).iterate(iter,newmodiscoll)
        newimg = image.set('closehumid',modis_close_collection)
        return newimg

      matched_landsatr = ee.ImageCollection(landsat_withmodis).map(datematch)
      matched_landsat = ee.ImageCollection(matched_landsatr).map(datematch_humid)

      # sweetpoint = ee.ImageCollection(landsat_withmodis).map(Strict_datematch)
      # // print("date near landsat",matched_landsat);
      # // print("sweetpoint :",sweetpoint);



      def modis_intersection(image):
        close_modis = ee.List(image.get('closemodis'))
        close_modis_inter = ee.List([])
        image_geom = image.geometry()
        def intersection_iter(img, close_modis_inter):
          modi_img_geom = ee.Image(img).geometry()
          intersected_geom = image_geom.intersection(modi_img_geom)
          modi_img_intersected = ee.Image(img).clip(intersected_geom)
          return ee.List(close_modis_inter).add(modi_img_intersected)

        close_modis_intersected = close_modis.iterate(intersection_iter,close_modis_inter)
        newimg = image.set('close_modis_inter',close_modis_intersected)
        modis_size = ee.List(close_modis_intersected).size()
        newimg = newimg.set('modis_inter_size', modis_size)
        return newimg

      def humid_intersection(image):
        close_modis = ee.List(image.get('closehumid'))
        close_modis_inter = ee.List([])
        image_geom = image.geometry()
        def intersection_iter(img, close_modis_inter):
          modi_img_geom = ee.Image(img).geometry()
          intersected_geom = image_geom.intersection(modi_img_geom)
          modi_img_intersected = ee.Image(img).clip(intersected_geom)
          return ee.List(close_modis_inter).add(modi_img_intersected)

        close_modis_intersected = close_modis.iterate(intersection_iter,close_modis_inter)
        newimg = image.set('close_humid_inter',close_modis_intersected)
        return newimg


      landsat_modisintersectedr = matched_landsat.map(modis_intersection)
      landsat_modisintersected = landsat_modisintersectedr.map(humid_intersection)
      # print('landsat with near date modis and intersected',landsat_modisintersected);

      landsat_modisintersected = landsat_modisintersected.filter(ee.Filter.gt('modis_inter_size', 0))
      # print('landsat with near date applying filter',landsat_modisintersected);
      # // Contains only images which have intersection with input file
      # // Contains the intersected landsat as property
      # // Contains the intesected modis as property
      # // Contains only images which have close modis date
      # // Map.addLayer(landsat_modisintersected);


      # // -------------------------- UPSCALING -----------------------------
      def calculate(image):
        ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
        savi = image.expression("((NIR-R)/(NIR+R+L))*(1+L)",{
          "NIR": image.select('SR_B4'),
          "R": image.select('SR_B3'),
          "L": 0.5
        }).rename("SAVI")
        ndbi = image.expression("(SWIR - NIR)/(SWIR+NIR)", {
          "SWIR": image.select('SR_B5'),
          "NIR": image.select('SR_B4')
        }).rename("NDBI")
        ndwi = image.expression("(green-NIR)/(green+NIR)", {
          "green": image.select('SR_B2'),
          "NIR": image.select('SR_B4')
        }).rename("NDWI")
        ndmi = image.expression("(NIR-SWIR1)/(NIR+SWIR1)", {
          "NIR":image.select('SR_B4'),
          "SWIR1": image.select('SR_B5')
        }).rename("NDMI")
        ndiib7 = image.expression("(NIR-SWIR2)/(NIR+SWIR2)",{
          "NIR":image.select('SR_B4'),
          "SWIR2": image.select('SR_B7')
        }).rename("NDIIB7")
        msavi = image.expression("(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - R)) ) / 2", {
          "NIR":image.select('SR_B4'),
          "R":image.select('SR_B3')
        }).rename("MSAVI")
        lst = image.expression("K1 * BT + K2" ,{
          'BT': image.select('ST_B6'),
          'K1': 0.00341802,
          'K2': 149.0
        }).rename('LST')
        albedo = image.expression("((0.130*B2) + (0.373*B3) + (0.085*B4) + (0.072*B5) -0.018) / 1.016",{
        # // 'B1': image.select('SR_B1'),
        'B2': image.select('SR_B1'),
        'B3': image.select('SR_B2'),
        'B4': image.select('SR_B3'),
        'B5': image.select('SR_B4'),
        }).rename('ALBEDO')
        return image.addBands([ndvi, savi, ndbi, ndmi, ndwi, ndiib7, msavi, albedo, lst])


      image_projection = landsat_modisintersected.first()
      # // print(image_projection, " This is projection 1");
      close_modis = ee.List(image_projection.get('close_modis_inter'))
      # // print(close_modis, " This is projection 2");
      close_modis_proj = ee.Image(close_modis.get(0))
      # print(close_modis_proj, " This is projection 3");
      proj = close_modis_proj.projection()
      # // print(proj, " This is projection 4")
      def processImage(image):
          return image.reduceResolution(
            reducer= ee.Reducer.mean(),
            bestEffort=True,
            maxPixels= 64
          ).reproject(
            crs= proj
          )

      landsat_orig_res = landsat_modisintersected.map(calculate)
      # print("landsat_orig_res :",landsat_orig_res)

      landsat_resampr = landsat_modisintersected.map(processImage).map(calculate)

      # ///////////// RESAMPLING DONE, NOW ADDING ET AND OTHER FEATURES AS BANDS /////////

      def add_humid(image):
        newimg = ee.List(image.get('close_humid_inter')).get(0)
        newimg2 = ee.Image(newimg).select('Qair_f_inst').reproject(proj).rename('humid')
        newimg3 = ee.Image(newimg).select('Rainf_tavg').reproject(proj).rename('rain')
        newimg4 = ee.Image(newimg).select('AvgSurfT_inst').reproject(proj).rename('temp')
        newimg5 = ee.Image(newimg).select('Psurf_f_inst').reproject(proj).rename('psurf')
        newimg6 = ee.Image(newimg).select('CanopInt_inst').reproject(proj).rename('canopy')
        newimg7 = ee.Image(newimg).select('SoilMoi0_10cm_inst').reproject(proj).rename('sm')
        newimg8 = ee.Image(newimg).select('Wind_f_inst').reproject(proj).rename('wind')
        newimg9 = ee.Image(newimg).select('RootMoist_inst').reproject(proj).rename('root')
        newimg10 = ee.Image(newimg).select('SoilTMP0_10cm_inst').reproject(proj).rename('soiltemp')
        newimg11 = ee.Image(newimg).select('Qsb_acc').reproject(proj).rename('runoff')
        newimg12 = ee.Image(newimg).select('SWdown_f_tavg').reproject(proj).rename('sw')
        newimg14 = ee.Image(newimg).select('Qle_tavg').reproject(proj).rename('qle')
        newimg15 = ee.Image(newimg).select('Qh_tavg').reproject(proj).rename('qh')
        newimg16 = ee.Image(newimg).select('Qg_tavg').reproject(proj).rename('qg')
        newimg17 = ee.Image(newimg).select('Swnet_tavg').reproject(proj).rename('swnet')
        newimg18 = ee.Image(newimg).select('Lwnet_tavg').reproject(proj).rename('lwnet')
        newimg19 = ee.Image(newimg).select('Tair_f_inst').reproject(proj).rename('tair')
        newimage = image.addBands([newimg2,newimg3,newimg4,newimg5,newimg6,newimg7,newimg8,newimg9,newimg10,newimg11,newimg12, newimg14, newimg15, newimg16, newimg17, newimg18, newimg19])
        return newimage

      def addBandET(image):
        modisimage_s = ee.Image(ee.List(image.get('close_modis_inter')).get(0)).select('ET')
        finalimage_s = image.addBands(modisimage_s)
        return finalimage_s


      def clipimage(image):
        clipped = ee.Image(image).clip(input_geom)
        return clipped

      # print("Landsat resampled :",landsat_resampr);

      landsat_500_a = landsat_resampr.map(add_humid)

      # print("Humid added: ", landsat_500_a.size());
      landsat_500 = landsat_500_a.map(addBandET)
      # print("ET added: ", landsat_500.size());
      landsat_30 = landsat_orig_res.map(add_humid)
      clipped_landsat_500 = landsat_500.map(clipimage)
      clipped_landsat_30 = landsat_30.map(clipimage)




      # //////////// BAND ADDED, GENERATING DATASET /////////////
      # /////////// USE GIVEN COLLECTIONS (landsat_500, landsat_30, clipped_landsat_500, clipped_landsat_30) accordingly.



      listsize = ee.Number(clipped_landsat_500.size());
      myList = ee.List.sequence(0, listsize.subtract(1));

      def extractTrainData(number):
          modellist_s = (clipped_landsat_500.toList(1,number))
          finalimage_s = ee.Image(modellist_s.get(0))
          trainingData2 = finalimage_s.sample(
            scale= 500,
            seed= 5,
            numPixels= 200)
          return trainingData2


      trainingList = myList.map(extractTrainData);


      def mergeCollections(collection, current):
        return ee.FeatureCollection(current).merge(ee.FeatureCollection(collection))


      trainingData = ee.FeatureCollection([])
      traindata_500 = ee.FeatureCollection(ee.List(trainingList).iterate(mergeCollections, trainingData))

      # //  dummyGeometry = ee.Geometry.Point([0, 0]);
      # //  TrainData = traindata_500.map(function(feature) {
      # //    geometry = dummyGeometry;
      # //   return feature.setGeometry(geometry);
      # // });

      # print("Train size: ", traindata_500.size());

      bands = ['MSAVI','NDMI','NDVI','NDWI','SAVI','NDBI','NDIIB7','ALBEDO','LST','humid','rain','temp','psurf','canopy','sm','wind','root','soiltemp', 'runoff', 'sw','qle','qh','qg','swnet','lwnet','tair','ET'];

      # Export to CSV
      print(filename)
      print(folder)
      task = ee.batch.Export.table.toDrive(
        collection = traindata_500,
        description = filename,
        selectors = bands,
        fileFormat = 'CSV',
        fileNamePrefix = filename,
        folder = folder
      )

      # Start the export task
      task.start()


      # ///////////// MOVE TO ANOTHER SCRIPT :(/Thread_ModelCreation/Model.js) ///////////////////

######################Starting:  0 #######################
index:  0  year:  2000   season:  1
2000_1
Train_Area_train_0
index:  0  year:  2000   season:  2
2000_2
Train_Area_train_0
index:  0  year:  2000   season:  3
2000_3
Train_Area_train_0
index:  0  year:  2000   season:  4
2000_4
Train_Area_train_0
index:  0  year:  2001   season:  1
2001_1
Train_Area_train_0
index:  0  year:  2001   season:  2
2001_2
Train_Area_train_0
index:  0  year:  2001   season:  3
2001_3
Train_Area_train_0
index:  0  year:  2001   season:  4
2001_4
Train_Area_train_0
index:  0  year:  2002   season:  1
2002_1
Train_Area_train_0
index:  0  year:  2002   season:  2
2002_2
Train_Area_train_0
index:  0  year:  2002   season:  3
2002_3
Train_Area_train_0
index:  0  year:  2002   season:  4
2002_4
Train_Area_train_0
index:  0  year:  2003   season:  1
2003_1
Train_Area_train_0
index:  0  year:  2003   season:  2
2003_2
Train_Area_train_0
index:  0  year:  2003   season:  3
2003_3
Train_Area_train_0
index:  0  

In [ ]:
# Export to CSV
print(filename)
print(folder)
task = ee.batch.Export.table.toDrive(
  collection = traindata_500,
  description = filename,
  selectors = bands,
  fileFormat = 'CSV',
  fileNamePrefix = filename,
  folder = folder
)

# Start the export task
task.start()


# Export.table.toDrive({
#   collection: traindata_500,
#   description: filename,
#   selectors:bands,
#   fileFormat: 'CSV',
#   fileNamePrefix: filename,
#   folder : folder
# });

# Map.addLayer(landsat_500);

2000_1
Train_Area_train_1
